In [1]:
import os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    folder_name = 'CSCI544-Project'
    folder_path = f'/content/drive/MyDrive/{folder_name}'
    os.chdir(folder_path)
    print("Current working directory is: " + os.getcwd())
    BASE_EMBEDDINGS_PATH = "./t5p_small_embeddings/"
except:
    BASE_EMBEDDINGS_PATH = "../t5p_small_embeddings/"
finally:
    print("Current working directory is: " + os.getcwd())
    print(os.listdir())

Current working directory is: /Users/swapnildhanwal/Documents/USC/CSCI 544/Project/repeat/src
['embeddings.ipynb', 'training.ipynb', 'model.py', 'utils.py', 'main.py', 'data.ipynb', 'replay.ipynb']


In [2]:
import torch

from model import FNNTopology, Model
from utils import load_tensors

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
X_train, y_train = load_tensors(BASE_EMBEDDINGS_PATH + "train/train_0.jsonl")
X_test, y_test = load_tensors(BASE_EMBEDDINGS_PATH + "test/test_0.jsonl")
print(X_train.shape)

torch.Size([7000, 256])


In [6]:
model = Model(FNNTopology("FNN with averaged vectors", X_train.shape[1]), 300, 100)

In [7]:
model.train(X_train, y_train)
model.evaluate(X_test, y_test)

Finished epoch 0, latest loss 0.689724
Finished epoch 1, latest loss 0.688570
Finished epoch 2, latest loss 0.688138
Finished epoch 3, latest loss 0.687244
Finished epoch 4, latest loss 0.687123
Finished epoch 5, latest loss 0.686107
Finished epoch 6, latest loss 0.686250
Finished epoch 7, latest loss 0.685295
Finished epoch 8, latest loss 0.685580
Finished epoch 9, latest loss 0.684431
Finished epoch 10, latest loss 0.684912
Finished epoch 11, latest loss 0.683534
Finished epoch 12, latest loss 0.683912
Finished epoch 13, latest loss 0.682368
Finished epoch 14, latest loss 0.682640
Finished epoch 15, latest loss 0.681039
Finished epoch 16, latest loss 0.680978
Finished epoch 17, latest loss 0.679482
Finished epoch 18, latest loss 0.679294
Finished epoch 19, latest loss 0.677711
Finished epoch 20, latest loss 0.677026
Finished epoch 21, latest loss 0.674360
Finished epoch 22, latest loss 0.672040
Finished epoch 23, latest loss 0.668042
Finished epoch 24, latest loss 0.663907
Finished e

(tensor(74.7452), tensor(0.7355))